# Day 14

In [1]:
import os

In [33]:
input_file_path = os.path.join(".", "day14.txt")
with open(input_file_path, 'r') as reader:
    input_data = reader.read()

## Part 1
Continuous sequence splicing

In [55]:
# Naive algorithm that tracks the actaul string
from collections import Counter

initial = None
instructions = {}

# Parse instructions
for line in input_data.split("\n"):
    if "->" in line:
        sequence, insertion = line.split(" -> ")
        instructions[sequence] = sequence[0] + insertion + sequence[1]
    elif initial is None:
        initial = line
    else:
        continue

# Process instructions
steps = 1
word = initial
for step in range(steps):
    sequences = []

    # Process each 2-gram
    for i, (a, b) in enumerate(zip(word[:-1], word[1:])):
        ch = a + b
        if ch in instructions:
            token = instructions[ch]
        
            # Account for overlaps
            if i > 0:
                token = token[1:]

        else:
            token = ch
        sequences.append(token)

    # Generate new word
    word = "".join(sequences)

counts = Counter(word).most_common()
delta = counts[0][1] - counts[-1][1]
print(f"Part 1: delta between most and least common element is {delta:,}")

counts

Part 1: delta between most and least common element is 6


[('V', 7),
 ('B', 5),
 ('H', 5),
 ('F', 5),
 ('P', 4),
 ('K', 4),
 ('S', 3),
 ('C', 3),
 ('N', 2),
 ('O', 1)]

## Part 2
Alternative algorithm to handle long mutations

In [63]:
# Alternative algorithm to only track letter occurances, not ordering
import numpy as np
from collections import defaultdict

initial = None
graph = {}
steps = 1

# Parse instructions
for line in input_data.split("\n"):
    if "->" in line:
        sequence, insertion = line.split(" -> ")
        expansion = sequence[0] + insertion + sequence[1]
        graph[sequence] = {expansion[:2], expansion[1:]}
    elif initial is None:
        initial = line
    elif len(line) > 0:
        raise RuntimeError(f"Unknown instruction: {line}")

# An array to indicate when we have x units of one sequence,
# what sequences we should receive after one step
columns = list(graph.keys())
sequence_cols = {k: i for i, k in enumerate(columns)}
n = len(columns)
arr_step = np.zeros((n, n))

# Populate transition matrix
for parent, (left, right) in graph.items():
    x = sequence_cols[parent]
    i = sequence_cols[left]
    j = sequence_cols[right]
    arr_step[x, [i, j]] += 1

# Initial vector
arr_initial = np.zeros(n)
arr_initial[[sequence_cols[x + y] for x, y in zip(initial[:-1], initial[1:])]] = 1

# Apply steps
arr = arr_initial @ np.linalg.matrix_power(arr_step, steps)

# Derive counts
counter = defaultdict(int)
for i, sequence in enumerate(columns):
    x, y = sequence
    count = int(arr[i])
    counter[x] += count
    counter[y] += count

# Adjust for overlap on first and last character
counter[initial[0]] += 1
counter[initial[-1]] += 1

# Difference between the most common and least common element
counts = sorted([(v // 2, k) for k, v in counter.items()])
delta = counts[-1][0] - counts[0][0]
print(f"Part 2: delta between most and least common element is {delta:,}")

counts

Part 2: delta between most and least common element is 5


[(1, 'O'),
 (2, 'N'),
 (3, 'C'),
 (3, 'F'),
 (3, 'S'),
 (4, 'K'),
 (4, 'P'),
 (5, 'B'),
 (5, 'H'),
 (6, 'V')]

In [7]:
# lows
# 3,265,991,822,211

In [102]:
import pandas as pd
df_new = pd.DataFrame(arr, index=columns, columns=["new"]).astype(int)
df_new.index.names = ["sequence"]

df_old = pd.DataFrame(
    data=Counter(x + y for x, y in zip(word[1:], word[:-1])).items(),
    columns=["sequence", "old"]
).set_index("sequence")

df_new.merge(df_old, left_index=True, right_index=True, how="outer").fillna(0).astype(int)


,new,old
sequence,,
BB,2,2
BC,1,0
BF,0,1
BH,0,0
BK,2,0
...,...,...
VN,0,1
VO,1,0
VP,0,2


,1
0,
NS,1
OV,1
PS,1
SK,1
KP,1
BN,1
NH,1
HF,1
VV,1
